In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE12295"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE12295"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE12295.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE12295.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE12295.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Class prediction models of thrombocytosis using genetic biomarkers"
!Series_summary	"Using custom spotted oligonucelotide platelet-focused arrays, platelet samples were compared. 48 health controls, 23 reactive thrombocytosis (RT) and 24 essential thrombocythemia (ET) samples were compared. An 11-biomarker gene subset was identified that discriminated among the three cohorts with 86.3% accuracy."
!Series_overall_design	"70 mer oligonucleotides were spotted in quadruplicate and hybridized versus reference RNA in two color method. Spiked control RNAs were also included."
Sample Characteristics Dictionary:
{0: ['Essential thrombocythemia Patient Platelet', 'Reactive Thrombocytosis Patient platelets', 'Normal Patient Platelets']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Determine gene expression data availability
# Based on series title and overall design, this appears to be a gene expression dataset
# using oligonucleotide arrays with platelet samples
is_gene_available = True

# Step 2: Clinical feature availability and type conversion
# From sample characteristics, we can see that row 0 contains information about patient type
# which includes "Essential thrombocythemia", "Reactive Thrombocytosis", and "Normal"
# This is the trait information we need for Essential_Thrombocythemia

# 2.1 Data Availability
trait_row = 0  # The trait information is in row 0
age_row = None  # No age information found
gender_row = None  # No gender information found

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait information to binary format (0 or 1)"""
    if isinstance(value, str):
        # Extract value after colon if it exists
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Check if "Essential thrombocythemia" is in the value
        if "Essential thrombocythemia" in value:
            return 1
        elif "Normal" in value or "Reactive Thrombocytosis" in value:
            return 0
    return None

def convert_age(value):
    """Convert age information to numeric format"""
    # Not needed since age data is not available
    return None

def convert_gender(value):
    """Convert gender information to binary format (0 for female, 1 for male)"""
    # Not needed since gender data is not available
    return None

# Step 3: Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Load the clinical data (assuming clinical_data is already loaded)
    try:
        clinical_data
    except NameError:
        print("Clinical data not loaded yet. Loading it now.")
        # This is a placeholder. In a real scenario, clinical_data would be loaded from a previous step
        # Since it's not provided, we'll create a placeholder to avoid errors
        clinical_data = pd.DataFrame()
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM309072': [1.0], 'GSM309073': [1.0], 'GSM309074': [1.0], 'GSM309075': [1.0], 'GSM309076': [1.0], 'GSM309077': [1.0], 'GSM309078': [1.0], 'GSM309079': [1.0], 'GSM309080': [1.0], 'GSM309081': [1.0], 'GSM309082': [1.0], 'GSM309083': [1.0], 'GSM309084': [1.0], 'GSM309085': [1.0], 'GSM309086': [1.0], 'GSM309087': [1.0], 'GSM309088': [1.0], 'GSM309089': [0.0], 'GSM309090': [1.0], 'GSM309091': [0.0], 'GSM309092': [1.0], 'GSM309093': [1.0], 'GSM309094': [1.0], 'GSM309095': [0.0], 'GSM309096': [0.0], 'GSM309097': [0.0], 'GSM309098': [0.0], 'GSM309099': [0.0], 'GSM309100': [0.0], 'GSM309101': [0.0], 'GSM309102': [0.0], 'GSM309103': [0.0], 'GSM309104': [0.0], 'GSM309105': [0.0], 'GSM309106': [0.0], 'GSM309107': [0.0], 'GSM309108': [0.0], 'GSM309109': [0.0], 'GSM309110': [0.0], 'GSM309111': [0.0], 'GSM309112': [0.0], 'GSM309113': [0.0], 'GSM309114': [0.0], 'GSM309115': [0.0], 'GSM309116': [0.0], 'GSM309117': [0.0], 'GSM309118': [0.0], 'GSM309119': [0.0], 'GSM30912

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009',
       '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018',
       '1019', '1020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers ('1001', '1002', etc.) are not human gene symbols.
# They appear to be numeric probe IDs that need to be mapped to gene symbols.
# Human gene symbols typically look like BRCA1, TP53, IL6, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1001', '1002', '1003', '1004', '1005'], 'ProbeID': ['1405_i_at', '1773_at', '200024_at', '200033_at', '200063_s_at'], 'Length': ['69', '69', '70', '70', '70'], 'TM': [76.668, 81.422, 78.843, 74.743, 74.743], 'Conc. (uM)': [50.0, 50.0, 50.0, 50.0, 50.0], 'Vol. (uL)': [200.0, 200.0, 200.0, 200.0, 200.0], 'SEQUENCE': ['AAAAGCTTCCCCAACTAAAGCCTAGAAGAGCTTCTGAGGCGCTGCTTTGTCAAAAGGAAGTCTCTAGGT', 'AGCTTAAGGATGAGACATCGGCAGAGCCTGCAACCGACTAGAGGACCTGGGTCCCGGCAGCTCTTTGCT', 'TCCTCGAACTCCTATGCCATTAAGAAGAAGGACGAGCTGGAGCGTGTGGCCAAGTCCAACCGCTGATTTT', 'TGCTACTGCAGCTGCACCTATGATTGGTTATCCAATGCCAACAGGATATTCCCAATAAGACTTTAGAAGT', 'AGAGTGAGAACTTTCCCTACCGTGTTTGATAAATGTTGTCCAGGTTCTATTGCCAAGAATGTGTTGTCCA'], 'Gene Symbol': ['CCL5', 'FNTB', '---', 'DDX5', 'NPM1'], 'ORF': ['CCL5', 'FNTB', nan, 'DDX5', 'NPM1'], 'SPOT_ID': [nan, nan, '200024_at', nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the annotation data
# From the preview, we can see that:
# - 'ID' in gene_annotation corresponds to gene identifiers in gene_data
# - 'Gene Symbol' contains the corresponding gene symbols

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the resulting gene expression data
print("Gene expression data preview after mapping:")
print(preview_df(gene_data))

# Print the shape of the resulting gene expression dataframe
print("Shape of gene expression data:", gene_data.shape)


Gene mapping preview:
{'ID': ['1001', '1002', '1003', '1004', '1005'], 'Gene': ['CCL5', 'FNTB', '---', 'DDX5', 'NPM1']}
Gene expression data preview after mapping:
{'GSM309072': [0.3613, 1.2385, 3.8373999999999997, 1.0297, 1.1287], 'GSM309073': [2.2002, 0.1765, 0.9791000000000001, -0.5281, 1.7099], 'GSM309074': [0.2529, 1.2249, 1.4834, 0.3542, 0.6599], 'GSM309075': [0.222, 1.194, 1.5643, 0.2396, 0.629], 'GSM309076': [0.0, -0.2831, 0.7523000000000001, 1.5443, 0.4921], 'GSM309077': [0.0, 0.371, -2.1963, 0.0623, -0.1802], 'GSM309078': [0.0, -0.9493, -1.4156, -1.1107, 0.0], 'GSM309079': [-3.2136, -0.3648, -0.7977, 0.0715, -0.5518], 'GSM309080': [0.0, -1.5601, -7.0526, -2.5894, -1.1948], 'GSM309081': [0.0, 1.6709, 4.8239, 0.8932, 1.7434], 'GSM309082': [-0.0265, 0.6641, 1.0421, -1.0281, -0.6279], 'GSM309083': [0.7839, -0.8244, -2.0099, -0.3996, 1.149], 'GSM309084': [-3.1558, -1.4091, -3.7744, -1.3303, -1.4615], 'GSM309085': [0.0, 0.0, 0.0, 1.0552, 0.0], 'GSM309086': [0.0, 0.0, 2.016, 0.0699,

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...


Gene data shape after normalization: (400, 95)


Normalized gene data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE12295.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM309072': [1.0], 'GSM309073': [1.0], 'GSM309074': [1.0], 'GSM309075': [1.0], 'GSM309076': [1.0], 'GSM309077': [1.0], 'GSM309078': [1.0], 'GSM309079': [1.0], 'GSM309080': [1.0], 'GSM309081': [1.0], 'GSM309082': [1.0], 'GSM309083': [1.0], 'GSM309084': [1.0], 'GSM309085': [1.0], 'GSM309086': [1.0], 'GSM309087': [1.0], 'GSM309088': [1.0], 'GSM309089': [0.0], 'GSM309090': [1.0], 'GSM309091': [0.0], 'GSM309092': [1.0], 'GSM309093': [1.0], 'GSM309094': [1.0], 'GSM309095': [0.0], 'GSM309096': [0.0], 'GSM309097': [0.0], 'GSM309098': [0.0], 'GSM309099': [0.0], 'GSM309100': [0.0], 'GSM309101': [0.0], 'GSM309102': [0.0], 'GSM309103': [0.0], 'GSM309104': [0.0], 'GSM309105': [0.0], 'GSM309106': [0.0], 'GSM309107': [0.0], 'GSM309108': [0.0], 'GSM309109': [0.0], 'GSM309110': [0.0], 'GSM309111': [0.0], 'GS

For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 24 occurrences. This represents 25.26% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Essential_Thrombocythemia/GSE12295.csv
